In [1]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import os
import time



In [2]:
data=pd.read_pickle('scrape_data.pkl')

In [3]:
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Journey
0,2007,1,1,1,1232.0,WN,SMF,ONT,389,SMFONT
1,2007,1,1,1,1918.0,WN,SMF,PDX,479,SMFPDX
6,2007,1,1,1,1936.0,WN,SMF,PHX,647,SMFPHX
12,2007,1,1,1,2128.0,WN,SMF,SAN,480,SMFSAN
25,2007,1,1,1,2119.0,WN,SMF,SEA,605,SMFSEA


In [29]:
data1=data[0:1]

In [16]:

def skyscannerscraper(dep,arr,date, direct='true'):
    # Function scraping skyscanner.dk
    # dep: airport code for departure airport
    # arr: airport code for arrival airport
    # date: date of flight, format YYMMDD
    # direct: indicates whether direct flight is preferred
    driver = webdriver.Firefox(executable_path='/Users/Christian/Dropbox/Universitet/TSODAS/test/geckodriver')
    driver.implicitly_wait(6)
    wait = WebDriverWait(driver, 12)
    #urlbase = 'https://www.skyscanner.dk/transport/fly/'
    #urlpas = '/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects='
    #urlend = '&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results'
    urlbase = 'https://www.skyscanner.com/transport/flights/'
    urlpas='/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects='
    urlend = '&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results'
    data = []

    for depa,arri in zip(dep,arr):
        url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date) + urlpas + direct + urlend


        driver.get(url)
            
        try:
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))
            
            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]


            data.append({
                "price": price[1],
                "duration": duration[1],
                "origin": str(depa),
                "destination": str(arri),
                "day":0
                    })
        except (TimeoutException,StaleElementReferenceException,BaseException):
            try:
                WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]
                
                data.append({
                "price": price[1],
                "duration": duration[1],
                "origin": str(depa),
                "destination": str(arri),
                "day":0
                    })    
            except (TimeoutException,StaleElementReferenceException,BaseException):
                try:
                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                    price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                    duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                    data.append({
                    "price": price[0],
                    "duration": duration[0],
                    "origin": str(depa),
                    "destination": str(arri),
                    "day":0
                        })  
                except (TimeoutException,StaleElementReferenceException,BaseException) :
                    try:
                        url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date+1) + urlpas + direct + urlend


                        driver.get(url)
                            

                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                        duration = [dur.text for dur in driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div/div[4]/section[3]/div/div[1]/div[3]/div/table/tbody/tr/td[2]/button/span[3]")]


                        data.append({
                                "price": price[1],
                                "duration": duration[1],
                                "origin": str(depa),
                                "destination": str(arri),
                                "day":1
                                    })
                    except (TimeoutException,StaleElementReferenceException,BaseException):
                        try:
                            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]

                            data.append({
                            "price": price[1],
                            "duration": duration[1],
                            "origin": str(depa),
                            "destination": str(arri),
                            "day":1
                                    })    
                        except (TimeoutException,StaleElementReferenceException,BaseException):  
                            try:
                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                                price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                                duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                                data.append({
                                "price": price[0],
                                "duration": duration[0],
                                "origin": str(depa),
                                "destination": str(arri),
                                "day":1
                                     }) 
                            except (TimeoutException,StaleElementReferenceException,BaseException):
                                try:
                                    url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date+2) + urlpas + direct + urlend


                                    driver.get(url)
                                        

                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                                    price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                                    duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]


                                    data.append({
                                            "price": price[1],
                                            "duration": duration[1],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":2
                                                })
                                except (TimeoutException,StaleElementReferenceException,BaseException):
                                    try:
                                        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                                        duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]

                                        data.append({
                                        "price": price[1],
                                        "duration": duration[1],
                                        "origin": str(depa),
                                        "destination": str(arri),
                                           "day":2
                                                })    
                                    except (TimeoutException,StaleElementReferenceException,BaseException):  
                                        try:
                                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                                            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                                            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                                            data.append({
                                            "price": price[0],
                                            "duration": duration[0],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":2
                                                 }) 
                                        except (TimeoutException,StaleElementReferenceException,BaseException): 
                                            try:
                                                url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date) + urlpas + direct + urlend


                                                driver.get(url)
                                                wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/section[3]/div/div[2]/div[2]/div[2]/dl[1]/dd/ol/li[3]/label/span[2]")))
                                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME,'CTASection__unpriced-label-29y5u')))
                                                price = [price.text for price in driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/section[3]/div/div[2]/div[2]/div[2]/dl[1]/dd/ol/li[3]/label/span[2]")]
                                                findes_d = [direk.text for direk in driver.find_elements(By.XPATH,"""//*[@id="16388-1905271000--31829-0-15040-1905271125"]/div/div[1]/div/div/div[2]/div[2]/div[2]/div/span[1]""")]
                                                data.append({
                                                "price": price,
                                                "duration": "-",
                                                "origin": str(depa),
                                                "destination": str(arri),
                                                "day":0,
                                                "findes_direkte":findes_d[0],
                                                "indirect":1
                                                     }) 
                                                
                                            except (TimeoutException,StaleElementReferenceException,BaseException):
                                                print("No working for "+str(url))
                                                data.append({
                                                "price": "-",
                                                "duration": "-",
                                                "origin": str(depa),
                                                "destination": str(arri),
                                                "day":"-"
                                                     }) 
        print(str(depa)+"-"+str(arri))
    driver.close()
                
    price_data = pd.DataFrame(data)                                            
    return price_data



In [40]:
start=time.time()
prices=skyscannerscraper(data1['Origin'],data1['Dest'],190527, direct='true')
stop =time.time()


SMF-ONT


In [35]:

def skyscannerscraper(dep,arr,date, direct='true'):
    # Function scraping skyscanner.dk
    # dep: airport code for departure airport
    # arr: airport code for arrival airport
    # date: date of flight, format YYMMDD
    # direct: indicates whether direct flight is preferred
    driver = webdriver.Firefox(executable_path='/Users/Christian/Dropbox/Universitet/TSODAS/test/geckodriver')
    driver.implicitly_wait(6)
    wait = WebDriverWait(driver, 12)
    #urlbase = 'https://www.skyscanner.dk/transport/fly/'
    #urlpas = '/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects='
    #urlend = '&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results'
    urlbase = 'https://www.skyscanner.com/transport/flights/'
    urlpas='/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects='
    urlend = '&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results'
    data = []

    for depa,arri in zip(dep,arr):
        url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date) + urlpas + direct + urlend


        driver.get(url)
            
        try:
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))
            
            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]


            data.append({
                "price": price[1],
                "duration": duration[1],
                "origin": str(depa),
                "destination": str(arri),
                "day":0
                    })
        except (TimeoutException,StaleElementReferenceException,BaseException):
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/section[3]/div/div[2]/div[2]/div[2]/dl[1]/dd/ol/li[3]/label/span[2]")))
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME,'CTASection__unpriced-label-29y5u')))
                price = [price.text for price in driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/section[3]/div/div[2]/div[2]/div[2]/dl[1]/dd/ol/li[3]/label/span[2]")]
                findes_d = [direk.text for direk in driver.find_elements(By.CLASS_NAME,"LegInfo__stops-label-1sfT2 LegInfo__stops-green-BoNBD")]
                data.append({
                        "price": price,
                        "duration": "-",
                        "origin": str(depa),
                        "destination": str(arri),
                        "day":0,
                        "findes_direkte":findes_d[0],
                        "indirect":1
                          }) 
                
            except (TimeoutException,StaleElementReferenceException,BaseException):
                    try:
                        url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date+1) + urlpas + direct + urlend


                        driver.get(url)
                            

                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                        duration = [dur.text for dur in driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div/div[4]/section[3]/div/div[1]/div[3]/div/table/tbody/tr/td[2]/button/span[3]")]


                        data.append({
                                "price": price[1],
                                "duration": duration[1],
                                "origin": str(depa),
                                "destination": str(arri),
                                "day":1
                                    })
                    except (TimeoutException,StaleElementReferenceException,BaseException):
                        try:
                            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]

                            data.append({
                            "price": price[1],
                            "duration": duration[1],
                            "origin": str(depa),
                            "destination": str(arri),
                            "day":1
                                    })    
                        except (TimeoutException,StaleElementReferenceException,BaseException):  
                            try:
                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                                price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                                duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                                data.append({
                                "price": price[0],
                                "duration": duration[0],
                                "origin": str(depa),
                                "destination": str(arri),
                                "day":1
                                     }) 
                            except (TimeoutException,StaleElementReferenceException,BaseException):
                                try:
                                    url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date+2) + urlpas + direct + urlend


                                    driver.get(url)
                                        

                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                                    price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                                    duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]


                                    data.append({
                                            "price": price[1],
                                            "duration": duration[1],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":2
                                                })
                                except (TimeoutException,StaleElementReferenceException,BaseException):
                                    try:
                                        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                                        duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]

                                        data.append({
                                        "price": price[1],
                                        "duration": duration[1],
                                        "origin": str(depa),
                                        "destination": str(arri),
                                           "day":2
                                                })    
                                    except (TimeoutException,StaleElementReferenceException,BaseException):  
                                        try:
                                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                                            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                                            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                                            data.append({
                                            "price": price[0],
                                            "duration": duration[0],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":2
                                                 }) 
                                        except (TimeoutException,StaleElementReferenceException,BaseException): 
                                            try:
                                                url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date) + urlpas + direct + urlend


                                                driver.get(url)
                                                wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/section[3]/div/div[2]/div[2]/div[2]/dl[1]/dd/ol/li[3]/label/span[2]")))
                                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME,'CTASection__unpriced-label-29y5u')))
                                                price = [price.text for price in driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/section[3]/div/div[2]/div[2]/div[2]/dl[1]/dd/ol/li[3]/label/span[2]")]
                                                findes_d = [direk.text for direk in driver.find_elements(By.XPATH,"""//*[@id="16388-1905271000--31829-0-15040-1905271125"]/div/div[1]/div/div/div[2]/div[2]/div[2]/div/span[1]""")]
                                                data.append({
                                                "price": price,
                                                "duration": "-",
                                                "origin": str(depa),
                                                "destination": str(arri),
                                                "day":0,
                                                "findes_direkte":findes_d[0],
                                                "indirect":1
                                                     }) 
                                                
                                            except (TimeoutException,StaleElementReferenceException,BaseException):
                                                print("No working for "+str(url))
                                                data.append({
                                                "price": "-",
                                                "duration": "-",
                                                "origin": str(depa),
                                                "destination": str(arri),
                                                "day":"-"
                                                     }) 
        print(str(depa)+"-"+str(arri))
    driver.close()
                
    price_data = pd.DataFrame(data)                                            
    return price_data

In [41]:
prices



,day,destination,duration,findes_direkte,indirect,origin,price
0,0,ONT,-,Direkte,1,SMF,[kr. 930]


In [81]:
def momondoscraper(dep,arr,yyyy,mm,dd, direct=0):
    # Function scraping skyscanner.dk
    # dep: airport code for departure airport
    # arr: airport code for arrival airport
    # date: date of flight, format YYMMDD
    # direct: indicates whether direct flight is preferred
    driver = webdriver.Firefox(executable_path='/Users/Christian/Dropbox/Universitet/TSODAS/test/geckodriver')
    driver.implicitly_wait(12)
    wait = WebDriverWait(driver, 12)
    urlbase='https://www.momondo.com/flight-search/'
    #https://www.momondo.com/flight-search/SMF-PHX/2019-06-06?sort=price_a&fs=stops=0
    #https://www.momondo.com/flight-search/SMF-PHX/2019-06-05?sort=price_a&fs=airlines=-WN;stops=0
    
    urlpas='?sort=duration_a&fs=stops='

    data = []

    for depa,arri in zip(dep,arr):
        url = urlbase + str(depa) + str('-') + str(arri) + str('/')+ str(yyyy)+str('-')+ str(mm)+str('-')+ str(dd) + urlpas + str(direct)


        driver.get(url)
            
        try:
            
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'display-price')))
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'section-duration')))
                                    
            price = [price.text for price in driver.find_elements(By.CLASS_NAME, 'display-price')]
            duration = [duration.text for duration in driver.find_elements(By.CLASS_NAME, 'section-duration')]


            data.append({
                "price": price[0],
                "duration": duration[0],
                "origin": str(depa),
                "destination": str(arri),
                "duration" : duration[0],
                "day":0
                    })
        except (TimeoutException,StaleElementReferenceException,BaseException):
                    try:
                        url = urlbase + str(depa) + str('-') + str(arri) + str('/')+ str(yyyy)+str('-')+ str(mm)+str('-')+ str(dd+1) + urlpas + str(direct)



                        driver.get(url)
                            
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'display-price')))
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'section-duration')))
                                    
                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, 'display-price')]
                        duration = [duration.text for duration in driver.find_elements(By.CLASS_NAME, 'section-duration')]

                        data.append({
                                "price": price[0],
                                "origin": str(depa),
                                "destination": str(arri),
                                "duration" : duration[0],
                                "day":1
                                    })
                    except (TimeoutException,StaleElementReferenceException,BaseException): 
                                try:
                                    url = urlbase + str(depa) + str('-') + str(arri) + str('/')+ str(yyyy)+str('-')+ str(mm)+str('-')+ str(dd+2) + urlpas + str(direct)



                                    driver.get(url)
                                        
                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'display-price')))
                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'section-duration')))
                                    
                                    price = [price.text for price in driver.find_elements(By.CLASS_NAME, 'display-price')]
                                    duration = [duration.text for duration in driver.find_elements(By.CLASS_NAME, 'section-duration')]


            

                                    data.append({
                                            "price": price[0],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "duration" : duration[0],
                                            "day":2
                                                })
                                except (TimeoutException,StaleElementReferenceException,BaseException):
                                    try:
                                        urlpas='?sort=duration_a&fs=airlines=-WN;stops='
                                        url = urlbase + str(depa) + str('-') + str(arri) + str('/')+ str(yyyy)+str('-')+ str(mm)+str('-')+ str(dd) + urlpas + str(direct)
                                        driver.get(url)
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'display-price')))
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'section-duration')))
                                    
                                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, 'display-price')]
                                        duration = [duration.text for duration in driver.find_elements(By.CLASS_NAME, 'section-duration')]
                                        data.append({
                                            "price": price[0],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":0,
                                            "duration" : duration[0],
                                            "direct":0
                                                })
                                    except (TimeoutException,StaleElementReferenceException,BaseException):
                                                print("No working for"+str(url))
                                                data.append({
                                                "price": "-",
                                                "origin": str(depa),
                                                "destination": str(arri),
                                                "duration" : duration[0],
                                                "day":"-"
                                                     }) 
        print(str(depa)+"-"+str(arri))
    driver.close()
                
    price_data = pd.DataFrame(data)                                            
    return price_data



In [82]:
price=momondoscraper(data1['Origin'],data1['Dest'],2019,10,28)

No working forhttps://www.momondo.com/flight-search/SMF-ONT/2019-10-28?sort=duration_a&fs=airlines=-WN;stops=0


UnboundLocalError: local variable 'duration' referenced before assignment

In [74]:
price

,day,destination,origin,price
0,-,ONT,SMF,-
1,-,PDX,SMF,-
2,-,PHX,SMF,-
